## Workflow Fundamentals

In [2]:
:dep candle-core = "0.8.1"
:dep candle-nn = "0.8.1"
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }

In [3]:
use candle_core::{DType, Device, Module, ModuleT, NdArray, Tensor, Var, D};
use candle_nn::linear::{Linear, linear};
use candle_nn::loss::mse;
use candle_nn::var_builder::{VarBuilder, SimpleBackend};
use candle_nn::{Optimizer, VarMap, SGD};
extern crate plotters;
use plotters::prelude::*;
use std::iter::{zip};


In [4]:
use candle_core::{Device, Tensor, DType};
let device = Device::Cpu;


### Step 1.  Getting Data Ready

Let's create some data that can be used for linear regression.

One interesting change from the original
```python
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
```

Because the tensor operationsl return Result<Tensor>, you have unwrap results of the multiplication before adding the bias.

A way (there are probably better ways), to print the first ten items is to turn the tensor into a vector using once of the methods on the Tensor.

In [5]:
let device = Device::Cpu;
let weight = 0.7;
let bias = 0.3;

let start = 0.0;
let end = 1.0;
let step = 0.02;
let X = Tensor::arange_step(start, end, step, &device)?;
//Do the multiplication first
let y = ((weight * X.clone())? + bias)?;

let x_vec = X.clone().to_vec1::<f64>()?;
let y_vec = y.clone().to_vec1::<f64>()?;
println!("{:?}", &x_vec[..10]);
println!("{:?}", &y_vec[..10]);
let train_split: usize = (0.8 * x_vec.len() as f32) as usize;
println!("Train: {}, Test:{}\n", train_split, x_vec.len() - train_split as usize);

let X_train = Tensor::from_slice(&x_vec[..train_split], (train_split,1), &device)?;
let y_train = Tensor::from_slice(&y_vec[..train_split], (train_split, 1), &device)?;
let x_test = Tensor::from_slice(&x_vec[train_split..x_vec.len()], (x_vec.len() - train_split, 1), &device)?;
let y_test = Tensor::from_slice(&y_vec[train_split..y_vec.len()], (y_vec.len() - train_split, 1), &device)?;


[0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12000000000000001, 0.14, 0.16, 0.18]
[0.3, 0.314, 0.32799999999999996, 0.34199999999999997, 0.356, 0.37, 0.384, 0.398, 0.412, 0.426]
Train: 40, Test:10



### Visualizing the Data

For this we will use plotter-rs.  Candle Tensors can be converted to vecs of different dimensions, so creating data points should be pretty easy

In [6]:
let training_datapoints: Vec<(f64, f64)> = zip(X_train.squeeze(1)?.to_vec1::<f64>()?, y_train.squeeze(1)?.to_vec1::<f64>()?).collect();
let test_datapoints: Vec<(f64, f64)> = zip(x_test.squeeze(1)?.to_vec1::<f64>()?, y_test.squeeze(1)?.to_vec1::<f64>()?).collect();


In [7]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Training Data", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    chart.draw_series(training_datapoints
                      .iter()
                      .map(|(x,y)| Circle::new((*x, *y), 2, BLUE.filled())))?
                      .label("Training Data")
                      .legend(|(x,y)| Circle::new((x, y), 2, BLUE.filled()));; 
    
    chart.draw_series(test_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, GREEN.filled())))?
                  .label("Test Data")
                  .legend(|(x,y)| Circle::new((x, y), 2, GREEN.filled()));; 

    chart.configure_series_labels().position(SeriesLabelPosition::UpperLeft).border_style(BLACK).background_style(WHITE.mix(0.1)).draw()?;
    
    Ok(())
}).style("width: 60%")

Training Data
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Training Data
 
 
Test Data

### 2.  Build Model

Create a linear regression model using candle NN linear layer.  You can let candle initialize a linear layer using the ```linear``` function, which automatically setup a weight and bias, or you can use ```Linear::new``` and provide the weight and bias yourself.  

In this case we'll create the weight and bias ourselves since there are iterations of the notebook where the weights are randomly initialized such that they are already very close to the original data.  This way we can initialize it with something that is way off so we can see the loss decrease over each training epoch.

**Note**

You'll want to use Var to declare your initial weights and bias and then clone them throughout the cells of your notebook.

In [8]:
let w = Var::new(&[[-0.2439f64]], &Device::Cpu)?;
let b = Var::new(0.3170f64, &Device::Cpu)?;

let model = Linear::new(w.as_tensor().clone(), Some(b.as_tensor().clone()));

In [9]:
//Let's look at the initialized weight and bias
println!("Weight: {}", model.weight());
match model.clone().bias() {
    Some(b) => println!("Bias: {}", b),
    None => {}
};


Weight: [[-0.2439]]
Tensor[[1, 1], f64]
Bias: [0.3170]


Let's have the model make some predications without any training

In [10]:
let y_preds = model.forward(&x_test)?;

Tensor[[], f64]


In [11]:
let pred_datapoints: Vec<(f64, f64)> = zip(x_test.squeeze(1)?.to_vec1::<f64>()?, y_preds.squeeze(1)?.to_vec1::<f64>()?).collect();

In [12]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Predication with No Training", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    chart.draw_series(training_datapoints
                      .iter()
                      .map(|(x,y)| Circle::new((*x, *y), 2, BLUE.filled())))?
                      .label("Training Data")
                      .legend(|(x,y)| Circle::new((x, y), 2, BLUE.filled()));; 
    
    chart.draw_series(test_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, GREEN.filled())))?
                  .label("Test Data")
                  .legend(|(x,y)| Circle::new((x, y), 2, GREEN.filled()));; 

        chart.draw_series(pred_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, RED.filled())))?
                  .label("Predication")
                  .legend(|(x,y)| Circle::new((x, y), 2, RED.filled()));; 

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .border_style(BLACK)
        .background_style(WHITE.mix(0.1))
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Predication with No Training
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Training Data
 
 
Test Data
 
 
Predication

Let's train the model now and see what happens

In [13]:
let mut train_loss_values: Vec<f64> = vec![];
let mut test_loss_values: Vec<f64> = vec![];
let mut epoch_count: Vec<f64> = vec![];
//Initialize the optimizer with the model weights and biases; be sure to clone
let mut opt = SGD::new(vec![w.clone(), b.clone()], 0.01)?;

for epoch in 0..3000 {
    //1.  Forward pass on the training data
    let y_preds = model.forward(&X_train)?;

    //2.  Calculate the loss, for this we will use Mean Squared Error (mse) from candle
    let loss = mse(&y_preds, &y_train)?;
    
    //3.  Candle does not have a zero_grad mechanism
    //4.  Get a GradStore from the loss
    let bp = loss.backward()?;
    //5. progress the optimzer
    opt.step(&bp);

    //Test the model
    let test_pred = model.forward(&x_test)?;
    let test_loss = mse(&test_pred, &y_test)?;
    
    if epoch % 10 == 0 {
        epoch_count.push(epoch as f64);
        train_loss_values.push(loss.to_scalar::<f64>()?);
        test_loss_values.push(test_loss.to_scalar::<f64>()?);
    }
}


()

In [14]:
// println!("Epochs: {:?}", epoch_count);
// println!("Training Loss Values: {:?}", train_loss_values);
// println!("Test Loss Values: {:?}", test_loss_values);

let train_loss_datapoints: Vec<(f64, f64)> = zip(epoch_count.clone(), train_loss_values.clone()).collect();
let test_loss_datapoints: Vec<(f64, f64)> = zip(epoch_count.clone(), test_loss_values.clone()).collect();
//println!("Train Loss Data Points: {:?}", train_loss_datapoints);
let max_tl: f64 = *test_loss_values.first().unwrap() + 0.05;
let max_epoch: f64 = *epoch_count.last().unwrap();


In [15]:


evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Training and Test Loss curves", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..max_epoch, 0f64..max_tl)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    //chart.draw_series(LineSeries::new(x_values.into_iter().map(|x| (x, 0.3 * x)), BLACK)).unwrap();
    chart.draw_series(LineSeries::new(train_loss_datapoints.clone().into_iter().map(|(x,y)| (x, y)), BLUE))
        .unwrap()
        .label("Training Loss")
        .legend(|(x,y)| Rectangle::new([(x - 10, y + 1), (x, y)], BLUE));
    chart.draw_series(LineSeries::new(test_loss_datapoints.clone().into_iter().map(|(x,y)| (x, y)), RED))
        .unwrap()
        .label("Test Loss")
        .legend(|(x,y)| Rectangle::new([(x - 10, y + 1), (x, y)], RED));

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperRight)
        .border_style(BLACK)
        .background_style(WHITE.mix(0.1))
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Training and Test Loss curves
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
 
0.0
 
 
 
500.0
 
 
 
1000.0
 
 
 
1500.0
 
 
 
2000.0
 
 
 
2500.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="47,314 48,367 50,384 52,396 54,403 56,408 58,411 60,413 62,414 64,415 66,416 68,416 70,417 72,417 74,418 76,418 78,418 80,418 82,419 84,419 86,419 88,419 90,420 92,420 93,420 95,420 97,421 99,421 101,421 103,421 105,421 107,422 109,422 111,422 113,422 115,422 117,423 119,423 121,423 123,423 125,423 127,423 129,424 131,424 133,424 135,424 137,424 138,424 140,425 142,425 144,425 146,425 148,425 150,425 152,425 154,426 156,426 158,426 160,426 162,426 164,426 166,426 168,426 170,426 172,427 174,427 176,427 178,427 180,427 182,427 183,427 185,427 187,427 189,427 191,428 193,428 195,428 197,428 199,428 201,428 203,428 205,428 207,428 209,428 211,428 213,428 215,429 217,429 219,429 221,429 223,429 225,429 227,429 228,429 230,429 232,429 234,429 236,429 238,429 240,429 242,429 244,429 246,430 248,430 250,430 252,430 254,430 256,430 258,430 260,430 262,430 264,430 266,430 268,430 270,430 272,430 273,430 275,430 277,430 279,430 281,430 283,430 285,430 287,430 289,431 291,431 293,431 295,431 297,431 299,431 301,431 303,431 305,431 307,431 309,431 311,431 313,431 315,431 317,431 318,431 320,431 322,431 324,431 326,431 328,431 330,431 332,431 334,431 336,431 338,431 340,431 342,431 344,431 346,431 348,431 350,431 352,431 354,431 356,431 358,431 360,431 362,432 363,432 365,432 367,432 369,432 371,432 373,432 375,432 377,432 379,432 381,432 383,432 385,432 387,432 389,432 391,432 393,432 395,432 397,432 399,432 401,432 403,432 405,432 407,432 408,432 410,432 412,432 414,432 416,432 418,432 420,432 422,432 424,432 426,432 428,432 430,432 432,432 434,432 436,432 438,432 440,432 442,432 444,432 446,432 448,432 450,432 452,432 453,432 455,432 457,432 459,432 461,432 463,432 465,432 467,432 469,432 471,432 473,432 475,432 477,432 479,432 481,432 483,432 485,432 487,432 489,432 491,432 493,432 495,432 497,432 498,432 500,432 502,432 504,432 506,432 508,432 510,432 512,432 514,432 516,432 518,432 520,432 522,432 524,432 526,432 528,432 530,432 532,432 534,432 536,432 538,432 540,432 542,432 543,432 545,432 547,432 549,432 551,432 553,432 555,432 557,432 559,432 561,432 563,432 565,432 567,432 569,432 571,432 573,432 575,432 577,432 579,432 581,432 583,432 585,432 587,432 588,432 590,432 592,432 594,432 596,432 598,432 600,432 602,432 604,432 606,432 608,432 610,432 612,432 614,432 616,432 618,432 620,432 622,432 624,432 626,432 628,432 630,432 632,432 "/>
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="47,68 48,146 50,201 52,241 54,271 56,293 58,309 60,322 62,332 64,340 66,346 68,351 70,355 72,358 74,361 76,364 78,366 80,368 82,370 84,371 86,373 88,374 90,376 92,377 93,378 95,379 97,380 99,381 101,382 103,383 105,384 107,385 109,386 111,387 113,387 115,388 117,389 119,390 121,391 123,391 125,392 127,393 129,394 131,394 133,395 135,396 137,396 138,397 140,398 142,398 144,399 146,400 148,400 150,401 152,401 154,402 156,402 158,403 160,404 162,404 164,405 166,405 168,406 170,406 172,407 174,407 176,408 178,408 180,408 182,409 183,409 185,410 187,410 189,411 191,411 193,411 195,412 197,412 199,412 201,413 203,413 205,414 207,414 209,414 211,415 213,415 215,415 217,416 219,416 221,416 223,416 225,417 227,417 228,417 230,418 232,418 234,418 236,418 238,419 240,419 242,419 244,419 246,420 248,420 250,420 252,420 254,421 256,421 258,421 260,421 262,421 264,422 266,422 268,422 270,422 272,422 273,423 275,423 277,423 279,423 281,423 283,423 285,424 287,424 289,424 291,424 293,424 295,424 297,425 299,425 301,425 303,425 305,425 307,425 309,425 311,425 313,426 315,426 317,426 318,426 320,426 322,426 324,426 326,426 328,427 330,427 332,427 334,427 336,427 338,427 340,427 342,427 344,427 346,427 348,428 350,428 352,428 354,428 356,428 358,428 360,428 362,428 363,428 365,428 367,428 369,428 371

Let's see how the model is doing after 100 epochs

In [16]:
let y_preds_after_training = model.forward(&x_test)?;

In [17]:
let pred_after_training_datapoints: Vec<(f64, f64)> = zip(x_test.squeeze(1)?.to_vec1::<f64>()?, y_preds_after_training.squeeze(1)?.to_vec1::<f64>()?).collect();

In [18]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Predication with 100 Epochs", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    chart.draw_series(training_datapoints
                      .iter()
                      .map(|(x,y)| Circle::new((*x, *y), 2, BLUE.filled())))?
                      .label("Training Data")
                      .legend(|(x,y)| Circle::new((x, y), 2, BLUE.filled()));; 
    
    chart.draw_series(test_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, GREEN.filled())))?
                  .label("Test Data")
                  .legend(|(x,y)| Circle::new((x, y), 2, GREEN.filled()));; 

    chart.draw_series(pred_datapoints
              .iter()
              .map(|(x,y)| Circle::new((*x, *y), 2, RED.filled())))?
              .label("Original Predication")
              .legend(|(x,y)| Circle::new((x, y), 2, RED.filled()));; 

    chart.draw_series(pred_after_training_datapoints
          .iter()
          .map(|(x,y)| Circle::new((*x, *y), 2, BLACK.filled())))?
          .label("Predication Post Training")
          .legend(|(x,y)| Circle::new((x, y), 2, BLACK.filled()));; 

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .border_style(BLACK)
        .background_style(WHITE.mix(0.1))
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Predication with 100 Epochs
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Training Data
 
 
Test Data
 
 
Original Predication
 
 
Predication Post Training

In [19]:
println!("Weight: {}", model.weight());
match model.clone().bias() {
    Some(b) => println!("Bias: {}", b),
    None => {}
};


Weight: [[0.6545]]
Tensor[[1, 1], f64]
Bias: [0.3186]
Tensor[[], f64]
